In [1]:
import requests
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    branca-0.3.0               |             py_0          24 KB  conda-forge
    altair-2.2.2               |           py36_1         461 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         558 KB

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py36_1      conda-forge
    branca:  0.3.0-py_0        conda-forge
    folium:  0.5.0-py_0        conda-forge
    vincent: 0.4.4-py_1        conda-forge

The following packages will be UPDATED:

    conda:

# Postal code from wikipedia

In [2]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'lxml')
My_table = soup.find('table',{'class':'wikitable sortable'})
labels=[]
data=[]
for tx in My_table.find_all('th'):
         labels.append(tx.text.strip())
rows = My_table.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    if cols:
     data.append([ele for ele in cols if ele])
    
df = pd.DataFrame.from_records(data, columns=labels)
df.rename(columns={'Postcode': 'PostalCode'}, inplace=True)
df.head()

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


# removing things that are not assigned

In [3]:
df1=df[df.Borough != 'Not assigned']
df1.shape

(212, 3)

# Update the unassigned

In [4]:
for index, row in df1.iterrows():
     if (row['Neighbourhood'] == 'Not assigned'):
        row['Neighbourhood'] = row['Borough']

# Geo-coordinates for different postal codes

In [8]:
import sys
import types




import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.
client_63009217aad749368c287a2b914cbff9 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='GvA3fdd2BSZEgS-8td33u1F-z92vvb-T3lTnAKf-C-e5',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_63009217aad749368c287a2b914cbff9.get_object(Bucket='datavisual-donotdelete-pr-2gfq67enauernl',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

geodata = pd.read_csv(body)
geodata.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
result = pd.merge(df1,geodata,on="PostalCode")
result.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
4,M6A,North York,Lawrence Heights,43.718518,-79.464763


# K-mean clustering forming 4 loaction clusters


In [12]:
# import k-means from clustering stage
from sklearn.cluster import KMeans
import folium # map rendering library
from urllib.parse import urljoin
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import numpy as np

df_grouped = result.groupby('Neighbourhood').mean().reset_index()

kclusters = 4

df_clustering = df_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

k_means_cluster_centers = kmeans.cluster_centers_

df_merged = df_clustering

# add clustering labels
df_merged['Cluster Labels'] = kmeans.labels_

# mapping clusters with different colour codes

In [14]:
latitude=43.6542599
longitude=-79.3606359

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

#set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

#print (folium.__version__)


# add markers to the map
#markers_colors = []
for lat, lon, cluster in zip(df_merged['Latitude'], df_merged['Longitude'], df_merged['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# dataset creation for the clusters

In [15]:
df_merged['Neighbourhood'] = df_grouped['Neighbourhood']
toronto_clustered = pd.merge(result,df_merged,on=['Neighbourhood','Latitude','Longitude'])
toronto_clustered.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels
0,M3A,North York,Parkwoods,43.753259,-79.329656,3
1,M4A,North York,Victoria Village,43.725882,-79.315572,3
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,0
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636,0
4,M6A,North York,Lawrence Heights,43.718518,-79.464763,1


# using four square location data set
# we pick top locations for the problem

In [17]:
CLIENT_ID = 'T100LAO1E4PPVOWA2HPGLEMG0W4DV22XNKGMHIDD0RKE0XOC' # your Foursquare ID
CLIENT_SECRET = 'IYD1RDNHDYF5LKFJDJSU54XMX2ECMIPO4KFD1KZEEZOJADZY' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT=100

def getNearbyVenues(names,latitudes,longitudes, PostalCode,Borough,ClusterLabels,radius=500):
    
    venues_list=[]
    for name, lat, lng,pc,borough,cluster in zip(names,latitudes,longitudes,PostalCode,Borough,ClusterLabels):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            pc,
            borough,
            cluster,
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood',
                  'PostalCode',
                  'Borough',
                  'ClusterLabels',
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

toronto_venues = getNearbyVenues(names=toronto_clustered['Neighbourhood'],
                                   latitudes=toronto_clustered['Latitude'],
                                   longitudes=toronto_clustered['Longitude'],
                                   PostalCode=toronto_clustered['PostalCode'],
                                   Borough=toronto_clustered['Borough'],
                                   ClusterLabels=toronto_clustered['Cluster Labels']
                                  )

Parkwoods
Victoria Village
Harbourfront
Regent Park
Lawrence Heights
Lawrence Manor
Queen's Park
Islington Avenue
Rouge
Malvern
Don Mills North
Woodbine Gardens
Parkview Hill
Ryerson
Garden District
Glencairn
Cloverdale
Islington
Martin Grove
Princess Gardens
West Deane Park
Highland Creek
Rouge Hill
Port Union
Flemingdon Park
Don Mills South
Woodbine Heights
Humewood-Cedarvale
Bloordale Gardens
Eringate
Markland Wood
Old Burnhamthorpe
Guildwood
Morningside
West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor
Downsview North
Wilson Heights
Thorncliffe Park
Adelaide
King
Richmond
Dovercourt Village
Dufferin
Scarborough Village
Fairview
Henry Farm
Oriole
Northwood Park
York University
East Toronto
Harbourfront East
Toronto Islands
Union Station
Little Portugal
Trinity
East Birchmount Park
Ionview
Kennedy Park
Bayview Village
CFB Toronto
Downsview East
The Danforth West
Riverdale
Design Exchange
Toronto

# venue counting foreach neighbour

In [18]:
toronto_venues.groupby(['Neighbourhood']).count()


,PostalCode,Borough,ClusterLabels,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,,,,
Adelaide,100,100,100,100,100,100,100,100,100
Agincourt,4,4,4,4,4,4,4,4,4
Agincourt North,3,3,3,3,3,3,3,3,3
Albion Gardens,11,11,11,11,11,11,11,11,11
Alderwood,10,10,10,10,10,10,10,10,10
Bathurst Manor,19,19,19,19,19,19,19,19,19
Bathurst Quay,14,14,14,14,14,14,14,14,14
Bayview Village,4,4,4,4,4,4,4,4,4
Beaumond Heights,11,11,11,11,11,11,11,11,11


 ## as we could see the large number of venues we need to reduce it to top 3 spots

In [19]:
Toronto_Zone1=toronto_venues.loc[toronto_venues['ClusterLabels'] == 0].groupby(['Neighbourhood']).count().sort_values(['Venue'],ascending=False)
Toronto_Zone1.head(20)

,PostalCode,Borough,ClusterLabels,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,,,,
Adelaide,100,100,100,100,100,100,100,100,100
First Canadian Place,100,100,100,100,100,100,100,100,100
Ryerson,100,100,100,100,100,100,100,100,100
Garden District,100,100,100,100,100,100,100,100,100
Toronto Dominion Centre,100,100,100,100,100,100,100,100,100
Design Exchange,100,100,100,100,100,100,100,100,100
Toronto Islands,100,100,100,100,100,100,100,100,100
Grange Park,100,100,100,100,100,100,100,100,100
Commerce Court,100,100,100,100,100,100,100,100,100


In [20]:
Toronto_food_zone1=toronto_venues.loc[toronto_venues['ClusterLabels'] == 0]
Toronto_food_zone1=Toronto_food_zone1[Toronto_food_zone1['Venue Category'].str.contains("food")]
Toronto_food_zone1.groupby(['Neighbourhood']).count().sort_values(['Venue'],ascending=False).head(5)

,PostalCode,Borough,ClusterLabels,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,,,,
Victoria Hotel,3,3,3,3,3,3,3,3,3
Commerce Court,3,3,3,3,3,3,3,3,3
Stn A PO Boxes 25 The Esplanade,3,3,3,3,3,3,3,3,3
Underground city,2,2,2,2,2,2,2,2,2
Davisville,2,2,2,2,2,2,2,2,2


# top 3 places in this cluster are vicroria hotel, commerce court, esplanade

# Now for the next cluster

In [22]:
toronto_venues.loc[toronto_venues['ClusterLabels'] == 1].groupby(['Neighbourhood']).count().sort_values(['Venue'],ascending=False)

,PostalCode,Borough,ClusterLabels,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,,,,
Oriole,62,62,62,62,62,62,62,62,62
Fairview,62,62,62,62,62,62,62,62,62
Henry Farm,62,62,62,62,62,62,62,62,62
Willowdale South,33,33,33,33,33,33,33,33,33
Bedford Park,25,25,25,25,25,25,25,25,25
Lawrence Manor East,25,25,25,25,25,25,25,25,25
Don Mills South,20,20,20,20,20,20,20,20,20
Flemingdon Park,20,20,20,20,20,20,20,20,20
Bathurst Manor,19,19,19,19,19,19,19,19,19


# In this top three regions is choosen i.e  oriole, fairview, henry farm

# cluster_3 we get

In [25]:
toronto_venues.loc[toronto_venues['ClusterLabels'] == 2].groupby(['Neighbourhood']).count().sort_values(['Venue'],ascending=False)

,PostalCode,Borough,ClusterLabels,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,,,,
Swansea,38,38,38,38,38,38,38,38,38
The Junction South,23,23,23,23,23,23,23,23,23
High Park,23,23,23,23,23,23,23,23,23
New Toronto,15,15,15,15,15,15,15,15,15
Mimico South,15,15,15,15,15,15,15,15,15
Humber Bay Shores,15,15,15,15,15,15,15,15,15
Mimico NW,12,12,12,12,12,12,12,12,12
The Queensway West,12,12,12,12,12,12,12,12,12
South of Bloor,12,12,12,12,12,12,12,12,12


# In this region too top 3 is choosen


# final cluster

In [29]:
toronto_venues.loc[toronto_venues['ClusterLabels'] == 3].groupby(['Neighbourhood']).count().sort_values(['Venue'],ascending=False)

,PostalCode,Borough,ClusterLabels,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,,,,
L'Amoreaux West,12,12,12,12,12,12,12,12,12
Steeles West,12,12,12,12,12,12,12,12,12
Woodbine Gardens,11,11,11,11,11,11,11,11,11
Parkview Hill,11,11,11,11,11,11,11,11,11
Golden Mile,10,10,10,10,10,10,10,10,10
Clairlea,10,10,10,10,10,10,10,10,10
Clarks Corners,10,10,10,10,10,10,10,10,10
Tam O'Shanter,10,10,10,10,10,10,10,10,10
Sullivan,10,10,10,10,10,10,10,10,10


# in this cluster only top 3 has more than 10 common points , so top 3 is choosen i.e  steelies west, woodbin gardens, L'Amoureax west

In [ ]:
# Thus , we got the top 3 locations in each of 4 clusters to start the store which might be profitable from